# Bitacora #5 - Combined_avg + Discount (Post-Paradoja)
**Fecha:** 2026-02-17  
**Autor:** Francis Bravo  

## 1. Motivacion

### Contexto de entrenamientos anteriores

**Bitácora #2 (Entrenamiento #1):**
- Parámetros: REWARD="propagate", LOSS="combined_avg", LR=5e-5, N_STATES=2
- Resultado: ❌ Q_select colapsado, Win Rate ~50% vs random

**Bitácora #3 (Entrenamiento #2):**
- Parámetros: REWARD="propagate", LOSS="combined_avg", LR=5e-4, N_STATES=6
- Resultado: ❌ Q_select colapsado, Win Rate ~45% vs random

**Bitácora #4 (Entrenamiento #1 ULTRA-AGRESIVO):**
- Parámetros: REWARD="discount", LOSS="only_select", LR=3e-4, N_STATES=8, 7 cambios simultáneos
- Resultado: ❌ Q_select colapsado, **pero descubrimiento crítico:**

### 🔴 Descubrimiento: La Paradoja del Only_Select

Al entrenar SOLO Q_select (`LOSS="only_select"`):
- Q_select (entrenada) → Colapsó a -1.0
- Q_place (NO entrenada) → **Mejoró significativamente** (aprendizaje pasivo del backbone)

**Implicación:** El entrenamiento conjunto (`combined_avg`) podría **estabilizar Q_select** aprovechando señales de Q_place.

### Hipótesis del experimento

**Si combinamos `REWARD_FUNCTION="discount"` (mejor señal temporal) con `LOSS_APPROACH="combined_avg"` (entrenamiento conjunto estabilizador), entonces Q_select NO colapsará y ambas cabezas aprenderán efectivamente.**

**Fundamento:**
1. `REWARD="discount"` demostró reducir loss 32% (bitácora 4)
2. Q_place aprendió mejor cuando el backbone se entrenaba con señales de Q_select
3. Entrenar ambas cabezas podría crear efecto estabilizador mutuo

## 2. Cambios de parametros

### 2.1 Estrategia seleccionada

**Estrategia: COMBINED_AVG + DISCOUNT (Post-Paradoja)**

Esta estrategia mantiene las mejoras de la bitácora #4 pero revierte el cambio de `LOSS_APPROACH` basándose en la Paradoja del Only_Select.

### 2.2 Cambios respecto a Bitácora #4 (Entrenamiento #1)

| Parámetro | Bitácora #4 | Bitácora #5 (NUEVO) | Razón del cambio |
|-----------|-------------|---------------------|------------------|
| **LOSS_APPROACH** | `"only_select"` | **`"combined_avg"`** | Aprovechar la Paradoja: entrenar ambas cabezas para estabilizar Q_select |

**Parámetros mantenidos de Bitácora #4:**
- ✅ REWARD_FUNCTION = "discount" (redujo loss 32%)
- ✅ LR = 3e-4 (estabilizó entrenamiento)
- ✅ GAMMA = 0.95 (necesario para discount)
- ✅ TAU = 0.01 (target network responsivo)
- ✅ N_LAST_STATES = 8 (mayor diversidad)
- ✅ TEMPERATURE_EXPLORE = 0.7 (mejor self-play)

### 2.3 Resumen de cambios vs Bitácora #2 (baseline original)

| Parámetro | Bitácora #2 (Baseline) | Bitácora #5 (NUEVO) | Diferencia |
|-----------|------------------------|---------------------|------------|
| **REWARD_FUNCTION** | `"propagate"` | **`"discount"`** | Señal temporal clara |
| **LOSS_APPROACH** | `"combined_avg"` | `"combined_avg"` | Sin cambio |
| **N_LAST_STATES** | 2/2 | **8/8** | 4x más diversidad |
| **TEMPERATURE_EXPLORE** | 2.0 | **0.7** | Mejor calidad |
| **TAU** | 0.01 | 0.01 | Sin cambio |
| **GAMMA** | 0.99 | **0.95** | Ajustado para discount |
| **LR** | 5e-5 | **3e-4** | 6x mayor |
| **BATCH_SIZE** | 30 | **64** | 2x mayor |

### 2.4 Parametros completos del entrenamiento

Parametros configurados para este experimento. Los valores **en negrita** son los que cambiaron respecto a la bitácora #2 (baseline original).

#### Configuracion principal

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| STARTING_NET | `None` | [L42](trainRL.py#L42) | [L45](trainRL_resume_latest.py#L45) | Pesos aleatorios (sin checkpoint previo) |
| EXPERIMENT_NAME | `"07_COMBINED_DISCOUNT"` | [L43](trainRL.py#L43) | [L46](trainRL_resume_latest.py#L46) | Nombre del experimento |
| ARCHITECTURE | `QuartoCNN_uncoupled` | [L50](trainRL.py#L50) | [L54](trainRL_resume_latest.py#L54) | Red con cabezas independientes (mantener) |
| LOSS_APPROACH | `"combined_avg"` | [L51](trainRL.py#L51) | [L55](trainRL_resume_latest.py#L55) | Promedio de Q_place y Q_select |
| **REWARD_FUNCTION** | **`"discount"`** | [L52](trainRL.py#L52) | [L56](trainRL_resume_latest.py#L56) | Recompensas descontadas temporalmente |

#### Hiperparametros de entrenamiento

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| EPOCHS | `3000` | [L66](trainRL.py#L66) | [L70](trainRL_resume_latest.py#L70) | Epocas totales |
| **BATCH_SIZE** | **`64`** | [L61](trainRL.py#L61) | [L65](trainRL_resume_latest.py#L65) | Muestras por batch (2x mayor que baseline) |
| **LR** | **`3e-4`** | [L111](trainRL.py#L111) | [L117](trainRL_resume_latest.py#L117) | Learning rate inicial (6x mayor que baseline) |
| LR_F | `1e-5` | [L112](trainRL.py#L112) | [L118](trainRL_resume_latest.py#L118) | Learning rate final |
| **TAU** | **`0.01`** | [L113](trainRL.py#L113) | [L119](trainRL_resume_latest.py#L119) | Tasa de soft update |
| **GAMMA** | **`0.95`** | [L115](trainRL.py#L115) | [L121](trainRL_resume_latest.py#L121) | Factor de descuento (ajustado para discount) |
| MAX_GRAD_NORM | `1.0` | [L110](trainRL.py#L110) | [L116](trainRL_resume_latest.py#L116) | Clipping de gradientes |
| torch.manual_seed | `5` | [L171](trainRL.py#L171) | [L204](trainRL_resume_latest.py#L204) | Semilla para reproducibilidad |

#### Generacion de experiencia

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| GEN_EXPERIENCE_BY_EPOCH | `True` | [L56](trainRL.py#L56) | [L60](trainRL_resume_latest.py#L60) | Generar experiencia nueva cada epoca |
| MATCHES_PER_EPOCH | `100` | [L73](trainRL.py#L73) | [L77](trainRL_resume_latest.py#L77) | Partidas de self-play por epoca |
| **N_LAST_STATES_INIT** | **`8`** | [L69](trainRL.py#L69) | [L73](trainRL_resume_latest.py#L73) | Estados del historial (inicio, 4x vs baseline) |
| **N_LAST_STATES_FINAL** | **`8`** | [L71](trainRL.py#L71) | [L75](trainRL_resume_latest.py#L75) | Estados del historial (final) |
| **TEMPERATURE_EXPLORE** | **`0.7`** | [L91](trainRL.py#L91) | [L95](trainRL_resume_latest.py#L95) | Temperatura para exploracion (vs 2.0 baseline) |
| TEMPERATURE_EXPLOIT | `0.1` | [L94](trainRL.py#L94) | [L98](trainRL_resume_latest.py#L98) | Temperatura para explotacion |
| mode_2x2 | `True` | [L63](trainRL.py#L63) | [L67](trainRL_resume_latest.py#L67) | Victoria por cuadrado 2x2 habilitada |

#### Evaluacion y guardado

| Parametro | Valor | trainRL | resume | Descripcion |
|-----------|-------|---------|--------|-------------|
| N_MATCHES_EVAL | `30` | [L59](trainRL.py#L59) | [L63](trainRL_resume_latest.py#L63) | Partidas de evaluacion vs baselines por epoca |
| FREQ_EPOCH_SAVING | `1000` | [L96](trainRL.py#L96) | [L100](trainRL_resume_latest.py#L100) | Guardar modelo cada N epocas |
| FREQ_EPOCH_PLOT_SHOW | `50` | [L101](trainRL.py#L101) | [L105](trainRL_resume_latest.py#L105) | Mostrar plots cada N epocas |
| PLOTS_FOLDER | `...\ Mech Interp\Plots` | [L48](trainRL.py#L48) | [L52](trainRL_resume_latest.py#L52) | Carpeta de guardado de plots HTML |
| SMOOTHING_WINDOW | `10` | [L104](trainRL.py#L104) | [L110](trainRL_resume_latest.py#L110) | Ventana de suavizado para plots |
| Q_PLOT_TYPE | `"hist"` | [L107](trainRL.py#L107) | [L113](trainRL_resume_latest.py#L113) | Tipo de plot de Q-values |

## 3. Valores derivados

Valores calculados automaticamente con los parametros:

| Valor | Calculo | Bitácora #2 | Bitácora #4 | Bitácora #5 |
|-------|---------|-------------|-------------|-------------|
| STEPS_PER_EPOCH | N_LAST_STATES_FINAL * MATCHES_PER_EPOCH | 2 * 100 = **200** | 8 * 100 = **800** | 8 * 100 = **800** |
| ITER_PER_EPOCH | STEPS_PER_EPOCH // BATCH_SIZE | 200 // 30 = **6** | 800 // 64 = **12** | 800 // 64 = **12** |
| REPLAY_SIZE | 100 * STEPS_PER_EPOCH | 100 * 200 = **20,000** | 100 * 800 = **80,000** | 100 * 800 = **80,000** |
| N_BATCHS_2_UPDATE_TARGET | ITER_PER_EPOCH // 3 | 6 // 3 = **2** | 12 // 3 = **4** | 12 // 3 = **4** |

### Comparación con experimentos anteriores:

**vs Bitácora #2 (baseline):**
- **4x más experiencia** por época (800 vs 200)
- **2x más iteraciones DQN** (12 vs 6)
- **4x más replay buffer** (80k vs 20k)
- **2x más updates del target** (4 vs 2)

**vs Bitácora #4:**
- **Mismos valores derivados** (cambio solo en LOSS_APPROACH)
- Permite comparación directa del efecto de combined_avg vs only_select

## 4. Registro de entrenamientos

<a id="train-table"></a>

| # | Fecha | Script | Epocas | Parámetros clave | Resultado | Ver gráficas |
|---|-------|--------|--------|-----------------|-----------|--------------||
| [1](#train-1) | 2026-02-17 | trainRL.py | 3000 | REWARD="discount", LOSS="combined_avg", N_STATES=8, TEMP=0.7, TAU=0.01, GAMMA=0.95, LR=3e-4 | 🔄 **EN PROGRESO** | ↓ |
| [2](#train-2) | - | - | - | - | - | ↓ |
| [3](#train-3) | - | - | - | - | - | ↓ |

---

### Notas sobre el entrenamiento #1

**Inicio:** 2026-02-17  
**Configuración:** COMBINED_AVG + DISCOUNT (Post-Paradoja)  
**Objetivo:** Aprovechar la Paradoja del Only_Select - entrenar ambas cabezas con mejor señal de recompensa  
**Duración esperada:** ~3000 épocas  
**Checkpoints:** Se guardarán en época 1000, 2000, 3000  

**Hipótesis a validar:**
1. Q_select NO colapsará (gracias al entrenamiento conjunto estabilizador)
2. Q_place mantendrá o mejorará su diferenciación clara
3. Win Rate superará 55% vs random
4. Win Rate vs bot_loss-BT superará 40%

**Métricas críticas a monitorear:**
- ✅ Q_select debe **mantener valores diversos** (no colapsar a -1.0)
- ✅ Q_select debe **diferenciar por recompensa** (R=-1 vs R=0 vs R=1)
- ✅ Q_place debe **mantener diferenciación** observada en bitácora #4
- ✅ Win Rate debe **superar 50%** vs random de forma consistente
- ✅ Loss debe **converger** (no solo oscilar)

**Comparación clave:**
Este entrenamiento es **casi idéntico** a Bitácora #4 excepto por `LOSS_APPROACH`. Esto permite aislar el efecto de entrenar ambas cabezas vs solo Q_select.

In [ ]:
from IPython.display import display, Image, Markdown
from os import path

IMG_FOLDER = r"C:\Users\bravo\Documents\Metodos Numericos Pycharm\Mech Interp\Imagenes de la Bitacoras"

# Descomentar cuando tengas las imágenes
# display(Markdown("#### Win Rate"))
# display(Image(filename=path.join(IMG_FOLDER, "WinRate_Bitacora5_Train1.png")))

# display(Markdown("#### Q-values Progress"))
# display(Image(filename=path.join(IMG_FOLDER, "QValueProgress_Bitacora5_Train1.png")))

# display(Markdown("#### Training Loss"))
# display(Image(filename=path.join(IMG_FOLDER, "TrainingLoss_Bitacora5_Train1.png")))

# display(Markdown("#### Board Comparisons"))
# display(Image(filename=path.join(IMG_FOLDER, "BoardComparison_Bitacora5_Train1.png")))

## 5. Observaciones y resultados

### 5.1 Entrenamiento #1 - COMBINED_AVG + DISCOUNT

<a id="train-1"></a>

**Fecha:** 2026-02-17  
**Status:** 🔄 EN PROGRESO

#### Parámetros usados:

```python
# Configuración crítica (cambio vs Bitácora #4)
REWARD_FUNCTION = "discount"      # Mantener de Bitácora #4
LOSS_APPROACH = "combined_avg"    # CAMBIO: de only_select a combined_avg
ARCHITECTURE = QuartoCNN_uncoupled  # Mantener

# Hiperparámetros (mantener de Bitácora #4)
LR = 3e-4
GAMMA = 0.95
TAU = 0.01
N_LAST_STATES = 8
TEMPERATURE_EXPLORE = 0.7
BATCH_SIZE = 64
```

#### Observaciones durante el entrenamiento:

**Época 0-500:**
- _completar observaciones tempranas_
- Q_select comportamiento: _completar_
- Q_place comportamiento: _completar_
- Comparación con Bitácora #4: _completar_

**Época 500-1500:**
- _completar observaciones intermedias_
- ¿Q_select mantiene valores diversos?: _completar_
- Convergencia del loss: _completar_

**Época 1500-3000:**
- _completar observaciones finales_
- Estabilidad final: _completar_

#### Resultados finales (Época 3000):

| Métrica | Valor | Bitácora #4 (only_select) | Cambio |
|---------|-------|---------------------------|--------|
| **Win Rate vs random** | _completar_ | ~47-50% | _completar_ |
| **Win Rate vs bot_loss-BT** | _completar_ | ~35-37% | _completar_ |
| **Q_select estado** | _completar_ | Colapsado a -1.0 | _completar_ |
| **Q_select diferenciación** | _completar_ | NO | _completar_ |
| **Q_place diferenciación** | _completar_ | SÍ - CLARA | _completar_ |
| **Loss final** | _completar_ | ~0.16-0.17 | _completar_ |

#### Análisis de gráficas:

**Win Rate:**
- _completar análisis_
- Comparación con Bitácora #4: _completar_

**Q-values Progress:**
- **Q_select:** _completar - ¿colapsó o se mantuvo?_
- **Q_place:** _completar - ¿mantuvo diferenciación?_
- **Efecto del combined_avg:** _completar_

**Training Loss:**
- _completar análisis de convergencia_
- Comparación con Bitácora #4: _completar_

**Board Comparisons:**
- _completar análisis_

#### Conclusión del entrenamiento #1:

**¿Se evitó el colapso de Q_select?**
- _completar: SÍ / NO / PARCIALMENTE_
- Evidencia: _completar_

**¿Mejoró vs Bitácora #4?**
- _completar comparación_

**¿Se validó la hipótesis?**
- _completar análisis_

**Efecto del cambio LOSS_APPROACH:**
- _completar - ¿combined_avg estabilizó Q_select?_

## 6. Conclusiones generales

### 6.1 Comparación con entrenamientos anteriores

| Métrica | Bitácora #2 | Bitácora #3 | Bitácora #4 | Bitácora #5 (este) |
|---------|-------------|-------------|-------------|--------------------|
| REWARD_FUNCTION | propagate | propagate | discount | **discount** |
| LOSS_APPROACH | combined_avg | combined_avg | only_select | **combined_avg** |
| Win Rate vs random | ~0.50 | ~0.45 | ~0.47-0.50 | _completar_ |
| Win Rate vs bot_loss-BT | ~0.37 | ~0.37 | ~0.35-0.37 | _completar_ |
| Q_select estado | Colapsado | Colapsado | Colapsado | _completar_ |
| Q_place diferenciación | No | Débil | Sí - CLARA | _completar_ |
| Loss final | ~0.25 | ~0.25 | ~0.16-0.17 | _completar_ |

### 6.2 Validación/Refutación de hipótesis

**Hipótesis:**
> "Si combinamos REWARD="discount" (mejor señal) con LOSS="combined_avg" (estabilización), entonces Q_select NO colapsará y ambas cabezas aprenderán efectivamente."

**Resultado:** _completar: VALIDADA / REFUTADA / PARCIALMENTE VALIDADA_

#### Análisis:

_completar análisis detallado_

### 6.3 Efecto del cambio LOSS_APPROACH

**Comparación directa Bitácora #4 vs #5:**

Ambos entrenamientos son idénticos excepto:
- Bitácora #4: `LOSS="only_select"`
- Bitácora #5: `LOSS="combined_avg"`

**Diferencias observadas:**

_completar tabla comparativa de métricas clave_

**Conclusión sobre LOSS_APPROACH:**

_completar - ¿combined_avg es mejor que only_select?_

### 6.4 Validación de la Paradoja del Only_Select

**Predicción basada en la paradoja:**
- Con `only_select`: Q_place mejoró, Q_select colapsó
- Con `combined_avg`: Ambas deberían mejorar o al menos Q_select no colapsar

**Resultado observado:**

_completar - ¿se confirmó la predicción?_

### 6.5 Próximos pasos

_completar basándose en los resultados obtenidos_

**Si Q_select NO colapsó:**
- Continuar optimizando hiperparámetros con esta configuración
- Probar entrenamientos más largos (5000+ épocas)
- Experimentar con shaped rewards

**Si Q_select SÍ colapsó:**
- Investigación arquitectural profunda (Opción 1 de Bitácora #4)
- Análisis de gradientes y pesos
- Considerar cambios arquitecturales menores

### 6.6 Conclusión final

_completar conclusión general del experimento_